// can't yet format YamlFrontmatter (["title: Scatter and line plots on Geo maps"; "category: Geo map charts"; "categoryindex: 6"; "index: 2"], Some { StartLine = 2 StartColumn = 0 EndLine = 6 EndColumn = 8 }) to pynb markdown



In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"


# Scatter and line plots on Geo maps

[![Binder](https://plotly.net/img/badge-binder.svg)](https://mybinder.org/v2/gh/plotly/Plotly.NET/gh-pages?filepath=5_1_geo-plots.ipynb)&emsp;
[![Script](https://plotly.net/img/badge-script.svg)](https://plotly.net/5_1_geo-plots.fsx)&emsp;
[![Notebook](https://plotly.net/img/badge-notebook.svg)](https://plotly.net/5_1_geo-plots.ipynb)

*Summary:* This example shows how to create Point and Line charts on geo maps in F#.

let's first create some data for the purpose of creating example charts:


In [3]:
open Plotly.NET 

let cityNames = [
    "Montreal"; "Toronto"; "Vancouver"; "Calgary"; "Edmonton";
    "Ottawa"; "Halifax"; "Victoria"; "Winnepeg"; "Regina"
]

let lon = [
    -73.57; -79.24; -123.06; -114.1; -113.28;
    -75.43; -63.57; -123.21; -97.13; -104.6
]
let lat = [
    45.5; 43.4; 49.13; 51.1; 53.34; 45.24;
    44.64; 48.25; 49.89; 50.45
]


The simplest type of geo plot is plotting the (lon,lat) pairs of a location via `Chart.PointGeo`. 
Here is an example using the location of Canadian cities:


In [4]:
let pointGeo =
    Chart.PointGeo(
        lon,
        lat,
        Labels=cityNames,
        TextPosition=StyleParam.TextPosition.TopCenter
    )
    |> Chart.withMapStyle(
        Scope=StyleParam.GeoScope.NorthAmerica, 
        Projection=GeoProjection.init(StyleParam.GeoProjectionType.AzimuthalEqualArea),
        CountryColor = "lightgrey"
    )
    |> Chart.withMarginSize(0,0,0,0)


In [ ]:
pointGeo


<!-- Plotly chart will be drawn inside this DIV -->

To connect the given (lon,lat) pairs via straight lines, use `Chart.LineGeo`. 
Below is an example that pulls external data as a Deedle data 
frame containing the source and target locations of American Airlines flights from Feb. 2011:


In [6]:
#r "nuget: Deedle"
#r "nuget: FSharp.Data"
open Deedle
open FSharp.Data
open System.IO
open System.Text

let data = 
    Http.RequestString "https://raw.githubusercontent.com/plotly/datasets/c34aaa0b1b3cddad335173cb7bc0181897201ee6/2011_february_aa_flight_paths.csv"
    |> fun csv -> Frame.ReadCsvString(csv,true,separators=",")

let opacityVals : float [] = data.["cnt"] |> Series.values |> fun s -> s |> Seq.map (fun v -> v/(Seq.max s)) |> Array.ofSeq
let startCoords = Series.zipInner data.["start_lon"] data.["start_lat"]
let endCoords = Series.zipInner data.["end_lon"] data.["end_lat"]
let coords = Series.zipInner startCoords endCoords |> Series.values

let flights = 
    coords 
    |> Seq.mapi (fun i (startCoords,endCoords) ->
        Chart.LineGeo(
            [startCoords; endCoords],
            Opacity = opacityVals.[i],
            Color = "red"
        )
    )
    |> Chart.Combine
    |> Chart.withLegend(false)
    |> Chart.withMapStyle(
        Scope=StyleParam.GeoScope.NorthAmerica, 
        Projection=GeoProjection.init(StyleParam.GeoProjectionType.AzimuthalEqualArea),
        ShowLand=true,
        LandColor = "lightgrey"
    )
    |> Chart.withMarginSize(0,0,50,0)
    |> Chart.withTitle "Feb. 2011 American Airline flights"


In [ ]:
flights


<!-- Plotly chart will be drawn inside this DIV -->